In [ ]:
from python_simulation_manager.monte_carlo_ising_for_rust import RustIsingExperiment, RustIsingExperimentBuilder, IsingData
import numpy as np
import matplotlib.pyplot as plt
from main import get_default_monte_carlo_parameters
import fssa

## Perform Monte-Carlo calculation:

In [ ]:
rust_dir = "../rust_simulation"
folder   = "results"
name     = "overview"


overview_builder = RustIsingExperimentBuilder(name=name, folder=folder, rust_dir=rust_dir)

In [ ]:
temperatures    = np.arange(0.5, 4.5, 0.05)
lengths         = [8, 16, 32, 64, 128]

(therm_steps, measure_steps) = get_default_monte_carlo_parameters(lengths)

overview = overview_builder.new_from_parameters(therm_steps=therm_steps, measure_steps=measure_steps, temperatures=temperatures, measure_struct_fact=False)

if not overview.are_parameter_files_available():
    overview.write_parameter_files() 

In [ ]:
print(f"\nLaunch the simulations: for {overview.get_lengths()}")
for L in overview.get_lengths():
    if not overview.has_output(L):
        overview.perform_rust_computation(L)

### Load from file & Plot!

In [ ]:
rust_dir  = "../rust_simulation"
folder    = "results"
name      = "overview"
lengths   = [8, 16, 32, 64, 128]

loader             = RustIsingExperimentBuilder(name=name, folder=folder, rust_dir=rust_dir)
overview_from_file = loader.load(lengths=lengths)

overview_results = overview_from_file.get_results()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,10))

ax1: plt.Axes = ax1
fig.suptitle(f"2D Ising using Swendsen-Wang", y=0.95, fontsize=20)
for (L, res) in overview_results.items():
    res: IsingData
    temps = res.temperatures
    
    ax1.scatter(temps, res.energy_density, s=15, marker= 'x', label=f"{L}x{L}")
    ax1.plot(temps, res.energy_density)#, label="Rust" )
    ax1.set_title(f"Energy density vs temp", fontsize=16)
    ax1.legend(loc="upper left",fontsize=14)
    # ax1.set_xlim((0.5,))
    
    ax2.scatter(temps, res.magnetisation, s=15, marker= 'x', label=f"{L}x{L}")
    ax2.plot(temps, res.magnetisation)#, label="Rust")
    ax2.set_title(f"Magnetization density vs temp", fontsize=16)
    ax2.legend(loc="upper right",fontsize=14)

    ax3.scatter(temps[1:], res.specific_heat[1:], s=15, marker= 'x', label=f"{L}x{L}")
    ax3.plot(temps[1:], res.specific_heat[1:])#, label="Rust")
    ax3.set_title(f"Specific heat/spin vs temp", fontsize=16)
    ax3.legend(loc="upper left",fontsize=14)

    ax4.scatter(temps, res.mag_susceptibility, s=15, marker= 'x', label=f"{L}x{L}")
    ax4.semilogy(temps, res.mag_susceptibility)
    ax4.set_title(f"Magnetic susceptibility/spin vs temp", fontsize=16)
    ax4.legend(loc="lower right",fontsize=14)

# Close to the critical temperature:

In [ ]:
rust_dir            = "../rust_simulation"
folder              = "results"
critical_experiment = "critical_temp"

critical_builder = RustIsingExperimentBuilder(name=critical_experiment, folder=folder, rust_dir=rust_dir)

In [ ]:
lengths_Tc = [8, 16, 32, 64, 128]

temps_Tc1 = np.array([1.9 + i*0.02 for i in range(10)])
temps_Tc2 = np.array([2.1 + i*0.01 for i in range(40)])
temps_Tc3 = np.array([2.5 + i*0.02 for i in range(14)]) #8 cores
temps_Tc  = np.concatenate((temps_Tc1, temps_Tc2, temps_Tc3))
print(f"For 8 cores: {len(temps_Tc)} temp values")

(therm_steps_Tc, measure_steps_Tc) = get_default_monte_carlo_parameters(lengths_Tc)


critical_exp   = critical_builder.new_from_parameters(therm_steps=therm_steps_Tc, measure_steps=measure_steps_Tc,temperatures=temps_Tc, measure_struct_fact=True)
if not critical_exp.are_parameter_files_available():
    critical_exp.write_parameter_files()

In [ ]:
for L in critical_exp.get_lengths():
    if not critical_exp.has_output(L):
        critical_exp.perform_rust_computation(L)

# Load from file & plot

In [ ]:
rust_dir            = "../rust_simulation"
folder              = "results"
critical_experiment = "critical_temp"

critical_builder = RustIsingExperimentBuilder(name=critical_experiment, folder=folder, rust_dir=rust_dir)
exp_critical     = critical_builder.load([8, 16, 32, 64, 128])
critical_results = exp_critical.get_results()

In [ ]:
plt.figure(figsize=(15,10))
for (L, result) in critical_results.items():
    result: IsingData
    temps = result.temperatures

    plt.scatter(temps, result.correlation_length/L, label=f"{L}x{L}")
    plt.semilogy(temps, result.correlation_length/L)
plt.legend()
plt.show()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,10))

plt.subplots_adjust(hspace=0.3)


fig.suptitle(f"2D Ising using the Swendsen-Wang algorithm", y=0.95, fontsize=20)
for (L, result) in critical_results.items():
    result: IsingData
    temps = result.temperatures
    
    
    ax1.scatter(temps, result.energy_density, s=15, marker= 'x', label=f"N={L}x{L}")
    ax1.plot(temps, result.energy_density) 
    ax1.set_title(f"Energy density vs temp", fontsize=17)
    ax1.legend(loc="upper left",fontsize=13)
    ax1.set_xlim(temps[0], temps[-1])
    ax1.set_ylim(np.min(result.energy_density), 0.95*np.max(result.energy_density))
    ax1.set_xlabel(r"Temperature [$J/k_B$]", fontsize=13)
    ax1.set_ylabel(r"Energy/spin [J]", fontsize=13)

    ax2.scatter(temps, result.magnetisation, s=15, marker= 'x', label=f"N={L}x{L}")
    ax2.plot(temps, result.magnetisation) 
    ax2.set_title(f"Magnetization", fontsize=17)
    ax2.legend(loc="lower left", fontsize=13)
    ax2.set_xlim(temps[0], temps[-1])
    ax2.set_ylim(0,1)
    ax2.set_xlabel(r"Temperature [$J/k_B$]", fontsize=13)
    ax2.set_ylabel(r"$|m|$", fontsize=13)

    ax3.scatter(temps, result.specific_heat, s=15, marker= 'x', label=f"N={L}x{L}")
    ax3.plot(temps, result.specific_heat) 
    ax3.set_title(f"Specific heat/Spin", fontsize=17)
    ax3.legend(loc="upper left",fontsize=13)
    ax3.set_xlim(temps[0], temps[-1])
    ax3.set_ylim(np.min(result.specific_heat), 1.05*np.max(result.specific_heat))
    ax3.set_xlabel(r"Temperature [$J/k_B$]", fontsize=13)
    ax3.set_ylabel(r"$C$", fontsize=13)

    ax4.scatter(temps, result.mag_susceptibility, s=15, marker= 'x', label=f"N={L}x{L}")
    ax4.semilogy(temps, result.mag_susceptibility)
    ax4.set_title(f"Magnetic susceptibility/Spin", fontsize=17)
    ax4.legend(loc="upper left",fontsize=13)
    ax4.set_xlabel(r"Temperature [$J/k_B$]", fontsize=13)
    ax4.set_ylabel(r"$\chi_M$", fontsize=13)
    ax4.set_xlim(temps[0], temps[-1])
    ax4.set_ylim(np.min(result.mag_susceptibility), 1.5*np.max(result.mag_susceptibility))


In [ ]:
lengths      = []
mag_suscept  = []
temps        = None
for (L, res) in critical_results.items():
    res: IsingData
    if temps is None:
        temps = res.temperatures
    
    lengths.append(L)
    mag_suscept.append(res.mag_susceptibility)
    
lengths     = np.asarray(lengths)
mag_suscept = np.asarray(mag_suscept)

In [ ]:
nu    = 1
zeta  = 1.75
rho_c = 2.27
a     = mag_suscept
da    = a * 0.1
ret   = fssa.autoscale(l=lengths, rho=temps, a=a, da=da, rho_c0=rho_c, nu0=1, zeta0=zeta)

In [ ]:
ret

In [ ]:
auto_scaled_data = fssa.scaledata(lengths, temps, a, da, ret.rho, ret.nu, ret.zeta)
print(ret.rho, ret.drho)
print(ret.nu, ret.dnu)
print(ret.zeta, ret.dzeta)
print(ret.fun)

In [ ]:

fig, ax = plt.subplots(figsize=(12,7))
plt.title(r"Data collapse of the magnetic susceptibility: $\nu={0:.3}$, $\zeta={1:.3}1$".format(ret.nu, ret.zeta),fontsize=17)
# ax.set_prop_cycle(cycler('color', palette))
max_y = 0
for (n, length) in enumerate(lengths):
    ax.scatter(auto_scaled_data.x[n], auto_scaled_data.y[n], label=f"L={length}x{length}", s=20, marker='x')

    max_data = np.max(auto_scaled_data.y[n])
    if max_data > max_y:
        max_y = max_data
  
ax.set_xlabel(r'$L^{1/\nu}(T-T_c)$')
ax.set_ylabel(r'$L^{-\zeta/{\nu}}\chi_M$')
plt.vlines(0, 0, 1.1*max_y, colors='k', linestyles=':', label=r"$T_c \approx {0:.4}$".format(ret.rho))
plt.xlim(-5, 5)
plt.ylim(0,1.1*max_y)
plt.legend(fontsize=16)
plt.show()

# Lets go to bigger sizes:

In [ ]:
rust_dir   = "../rust_simulation"
folder     = "results"
experiment = "size_limit"

size_builder = RustIsingExperimentBuilder(name=experiment, folder=folder, rust_dir=rust_dir)

In [ ]:
lengths      = [4, 8, 16, 32, 64, 128, 256, 512, 1024]
temperatures = np.array([1.7 + i*0.02 for i in range(56)])

(therm_steps_size, measure_steps_size) = get_default_monte_carlo_parameters(lengths)
            
size_exp = size_builder.new_from_parameters(therm_steps=therm_steps_size,measure_steps=measure_steps_size, temperatures=temperatures, measure_struct_fact=False)
# if not size_exp.are_parameter_files_available():
size_exp.write_parameter_files()

In [ ]:
for L in size_exp.lengths:
    size_exp.perform_rust_computation(L)

## Load

In [ ]:
rust_dir   = "../rust_simulation"
folder     = "results"
experiment = "size_limit"

size_exp_builder = RustIsingExperimentBuilder(name=experiment, folder=folder, rust_dir=rust_dir)
lengths          = [8, 16, 32, 64, 128, 256, 512, 1024]
size_exp         = size_exp_builder.load(lengths)

In [ ]:
size_results = size_exp.get_results()

In [ ]:
# fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,10))
plt.figure(figsize=(15,10))
plt.title(f"2D Ising: Scaling", y=0.95, fontsize=20)

temps = None
for (L, res) in size_results.items():
    res: IsingData
    if temps is None:
        temps = res.temperatures

    plt.plot(temps, res.magnetisation)
    plt.scatter(temps, res.magnetisation, s=15, marker= 'x', label=f"{L}x{L}")
plt.xlim(temps[0], temps[-1])
plt.ylim(0,1)
plt.xlabel(r"Temperature [$J/k_B$]", fontsize=13)
plt.ylabel(r"$|m|$", fontsize=13)
plt.legend(fontsize=16)